# Hands-on with the NextOcean API: A virtual tutorial
+ Author: Miguel Chapela Rivas, Deimos Space
+ Date: 13/09/2023

## Outline
1. Initialize connection and login
2. Get service list
3. Get Service information
4. Order service
5. Get order status
6. Download products

## 1- Initialize connection and login

In [1]:
# Import the required library
import request4EO
import yaml
import requests
import os 

# Initialize the connection with request4EO API
requestAPI = request4EO.requestServiceAPI()

# Load credentials from the 'credentials.yaml' file
credentials = yaml.safe_load(open('credentials.yaml', 'r'))

# Extract the username and password from the credentials file
username = credentials['username']
password = credentials['password']

# Use the extracted username and password to log in to the API
login = requestAPI.login(username, password)


## 2- Get service list

#### Use the function get_services_list(self) to return the list of all available services ids

In [ ]:
# Use function get_services_list with the object created before
services_list = requestAPI.get_services_list()
services_list

## 3- Get Service information

In [ ]:
# Use function get_services with the object created before
service_id = 'FPE:_FISHING_FOOTPRINT'
# service_id = 'CFA__POTENTIAL_FISHING_AREAS'
service = requestAPI.get_service(service_id)
service

### Get the service period and and footprint

In [4]:
# Extract servicePeriod
print('Minimum start date')
print(service['servicePeriod']['serviceStart'])
print('Maximum end date')
print(service['servicePeriod']['serviceStop'])
print()

# Extract serviceFootprint
# The vector featrure can be visualized in https://wktmap.com/
print('The service footprint is: ')
print(service['serviceFootprint'])


Minimum start date
2012-01-01T01:00:00
Maximum end date
2018-12-31T01:00:00

The service footprint is: 
MULTIPOLYGON(((-29.941406249999986 27.163067787973674,-22.382812499999986 29.483875765537963,-14.999999999999986 27.94228443709624,-9.462890624999986 32.20005534819621,-6.430664062499987 33.82140647652505,-5.595703124999987 36.338393838228555,-7.045898437499987 37.393106949944496,-8.441162109374986 37.40128947428784,-8.298339843749986 38.03514606300014,-8.847656249999986 38.87716915873357,-8.803710937499986 39.58984849551525,-8.408203124999986 41.29125390044174,-8.847656249999986 42.768218140889324,-13.769531249999986 43.02576472850212,-17.988281249999986 43.2502353481404,-21.328124999999986 44.17140053246825,-23.437499999999986 48.76798524483268,-32.050781249999986 49.57245856275113,-33.632812499999986 46.63909468774348,-42.949218749999986 45.418726354605006,-42.773437499999986 32.497057800813295,-32.578124999999986 30.397789896897468,-29.941406249999986 27.163067787973674)))


In [11]:
# Extract input identifiers and values
input_identifiers = [(item['identifier'], item['title']) for item in service['input']]

# Print the input identifiers
if not input_identifiers:
    print('The service has no addditional inputs')
else:    
    for identifier, title in input_identifiers:
        print(f"{identifier} -> {title}")


spatial_resolution -> Select the grid spatial resolution (km)
MMSI -> Maritime Mobile Service Identity (Select All for no filter or a single MMSI)
main_gear -> Fishing technique of the vessel
country -> Registration flag of the vessels
vessel_loa -> LOA (Vessel Length [meters])


In [6]:
# Extract input identifiers and values
input_identifiers = [(item['identifier'], item['literalData']['value']) for item in service['input']]
title
# Print the input identifiers
if not input_identifiers:
    print('The service has no addditional inputs')
else:      
    for identifier, value in input_identifiers:
        print(f"Identifier: {identifier}")
        print(f"Value: {value}")


Identifier: spatial_resolution
Value: ['50', '10', '5', '1']
Identifier: MMSI
Value: ['All', '111111115', '112233445', '201100145', '204041000', '204201240', '204201270', '204201370', '204201560', '204201740', '204202570', '204203250', '204203650', '204203880', '204204280', '204204560', '204204780', '204204950', '204205960', '204206740', '204206750', '204206760', '204207970', '204208670', '204208790', '204209570', '204209840', '204209870', '204209890', '204209950', '204209960', '204210770', '204211000', '204211450', '204213000', '204225000', '204226000', '204229000', '204232000', '204233000', '204236000', '204237000', '204242000', '204243000', '204244000', '204245000', '204248000', '204250000', '204255000', '204257000', '204258000', '204259000', '204260000', '204261000', '204263000', '204265000', '204266000', '204267000', '204268000', '204269000', '204270000', '204274000', '204275000', '204277000', '204278000', '204284000', '204285000', '204287000', '204807000', '204809000', '204813000

## 4- Order service

Function -> order_service(self,service_id, service_parameters) 
<pre>
service_parameters = {
    "aoi": "<area in WKT>",
    "beginAt": "<begin date in format YYYY-mm-ddTHH:MM>",
    "endAt": "<end date in format YYYY-mm-ddTHH:MM>",

}
    #Each service specific inputs should be formated in a list of dictionaries like this: 

    "inputs": [{
                    "identifier": "<parameter_identifier_1>",
                    "value": "<parameter_value_1>"
                    },
                {
                    "identifier":"<parameter_identifier_2>",
                    "value":"<parameter_value_2>"
                    }]
    }

</pre>

### Order CFA: Characterization of Fishing Areas - Potential Fishing Areas

In [90]:
from datetime import datetime
import shapely.wkt

aoi_str ="POLYGON ((-11.052246 40.615198, -8.76709 40.548466, -9.404297 37.337467, -10.678711 37.529337, -11.052246 40.615198))"
aoi = shapely.wkt.loads(aoi_str)
starting_date = datetime(2018,8,1)
final_date = datetime(2018,8,1)

service_parameters_CFA = {
    "aoi": aoi.wkt,
    "beginAt": starting_date.strftime("%Y-%m-%dT%H:%M:%S"),
    "endAt": final_date.strftime("%Y-%m-%dT%H:%M:%S"),
    "inputs": [] 
    }

service_parameters_CFA    

{'aoi': 'POLYGON ((-11.052246 40.615198, -8.76709 40.548466, -9.404297 37.337467, -10.678711 37.529337, -11.052246 40.615198))',
 'beginAt': '2018-08-01T00:00:00',
 'endAt': '2018-08-01T00:00:00',
 'inputs': []}

In [91]:
order_info_CFA = requestAPI.order_service('CFA__POTENTIAL_FISHING_AREAS', service_parameters_CFA)
order_info_CFA

{'id': 606417385,
 'uuid': '719486f4-58f7-4014-a9b1-cce77898649f',
 'status': 'CREATED',
 'serviceId': 'CFA__POTENTIAL_FISHING_AREAS',
 'inputFileTypes': ['GLO_SST_L4_NRT_OBS_SST_V2'],
 'userUuid': 'b6443660-eede-4ec7-b026-68d0097d2ab8',
 'aoi': 'POLYGON ((-11.052246 40.615198, -8.76709 40.548466, -9.404297 37.337467, -10.678711 37.529337, -11.052246 40.615198))',
 'beginAt': '2018-08-01T00:00:00',
 'endAt': '2018-08-01T00:00:00',
 'paymentTransaction': {'id': 606417433,
  'transactionReference': 'fe8680cf-af47-43fe-8f1c-44b3e6c9dd49',
  'status': 'PAID',
  'amount': 0.0,
  'paymentMethod': 'FREE',
  'createdAt': '2023-09-07T10:05:22.698979',
  'updatedAt': '2023-09-07T10:05:22.698984',
  'vat': 0.0},
 'inputs': [],
 'files': [{'id': 606423257,
   'orderId': 606417385,
   'file': '20180801120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB-v02_0-fv02_0'}],
 'createdAt': '2023-09-07T10:05:22.711466',
 'updatedAt': '2023-09-07T10:05:22.711468',
 'paymentProcessing': False,
 'created': True,
 'paymen

### Order FPE: Fishing provenance and ecolabelling - Fishing Footprint

In [89]:

aoi_str ="POLYGON((-11.335880496111539 39.499469670842466,-10.061466433611539 39.499469670842466,-10.061466433611539 38.57789510908854,-11.335880496111539 38.57789510908854,-11.335880496111539 39.499469670842466))"

aoi = shapely.wkt.loads(aoi_str)
starting_date = datetime(2018,8,1)
final_date = datetime(2018,8,15)

service_parameters_FPE_FF = {
    "aoi": aoi.wkt,
    "beginAt": starting_date.strftime("%Y-%m-%dT%H:%M:%S"),
    "endAt": final_date.strftime("%Y-%m-%dT%H:%M:%S"),
    "inputs": [
                {
                    "identifier": "spatial_resolution",
                    "value": "10"
                    },
                {
                    "identifier": "MMSI",
                    "value": "All"
                    },
                {
                    "identifier": "main_gear",
                    "value": "All"
                    },  
                 {
                    "identifier": "country",
                    "value": "All"
                    },        
                {
                    "identifier":"vessel_loa",
                    "value":"All"
                    }] 
    }

service_parameters_FPE_FF    

{'aoi': 'POLYGON ((-11.335880496111539 39.499469670842466, -10.061466433611539 39.499469670842466, -10.061466433611539 38.57789510908854, -11.335880496111539 38.57789510908854, -11.335880496111539 39.499469670842466))',
 'beginAt': '2018-08-01T00:00:00',
 'endAt': '2018-08-15T00:00:00',
 'inputs': [{'identifier': 'spatial_resolution', 'value': '10'},
  {'identifier': 'MMSI', 'value': 'All'},
  {'identifier': 'main_gear', 'value': 'All'},
  {'identifier': 'country', 'value': 'All'},
  {'identifier': 'vessel_loa', 'value': 'All'}]}

In [25]:
order_info_FPE_FF = requestAPI.order_service('FPE:_FISHING_FOOTPRINT', service_parameters_FPE_FF)
order_info_FPE_FF

{'id': 606417381,
 'uuid': '004bc596-4af1-436e-93f2-958c8d10db02',
 'status': 'CREATED',
 'serviceId': 'FPE:_FISHING_FOOTPRINT',
 'inputFileTypes': ['VESSEL_PATHS_AIS_NE_ATLANTIC'],
 'userUuid': 'b6443660-eede-4ec7-b026-68d0097d2ab8',
 'aoi': 'POLYGON ((-11.335880496111539 39.499469670842466, -10.061466433611539 39.499469670842466, -10.061466433611539 38.57789510908854, -11.335880496111539 38.57789510908854, -11.335880496111539 39.499469670842466))',
 'beginAt': '2018-08-01T00:00:00',
 'endAt': '2018-08-15T00:00:00',
 'paymentTransaction': {'id': 606417429,
  'transactionReference': 'acb2a03a-dc91-4f58-adde-a3bdb52764b9',
  'status': 'PAID',
  'amount': 0.0,
  'paymentMethod': 'FREE',
  'createdAt': '2023-09-07T08:11:49.164707',
  'updatedAt': '2023-09-07T08:11:49.164708',
  'vat': 0.0},
 'inputs': [{'id': 47594,
   'identifier': 'spatial_resolution',
   'dataType': 'string',
   'value': '10'},
  {'id': 47595, 'identifier': 'MMSI', 'dataType': 'string', 'value': 'All'},
  {'id': 47596,

## 5 - Get order status

In [78]:
order_id_CFA = order_info_CFA["id"]
order_info_CFA = requestAPI.get_order(order_id_CFA)
order_info_CFA

{'id': 606417384,
 'uuid': '0e53ccbc-e446-4f9f-b728-f354e3105b85',
 'status': 'FINISHED',
 'serviceId': 'CFA__POTENTIAL_FISHING_AREAS',
 'inputFileTypes': ['GLO_SST_L4_NRT_OBS_SST_V2'],
 'userUuid': 'b6443660-eede-4ec7-b026-68d0097d2ab8',
 'aoi': 'POLYGON ((-11.052246 40.615198, -8.76709 40.548466, -9.404297 37.337467, -10.678711 37.529337, -11.052246 40.615198))',
 'beginAt': '2018-08-01T00:00:00',
 'endAt': '2018-08-01T00:00:00',
 'paymentTransaction': {'id': 606417432,
  'transactionReference': 'b5f72b61-36ac-4aa8-bf83-8a615bf98b78',
  'status': 'PAID',
  'amount': 0.0,
  'paymentMethod': 'FREE',
  'createdAt': '2023-09-07T08:47:03.368971',
  'updatedAt': '2023-09-07T08:47:03.368974',
  'vat': 0.0},
 'inputs': [],
 'files': [{'id': 606423256,
   'orderId': 606417384,
   'file': '20180801120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB-v02_0-fv02_0'}],
 'createdAt': '2023-09-07T08:47:03.391551',
 'updatedAt': '2023-09-07T08:50:21.199128',
 'serviceType': 'Processing_Service',
 'serviceSubtype

In [64]:
print('The order {order_id} status is {order_status}'.format(order_id= order_id_CFA, order_status =order_info_CFA['status']))

The order 606417384 status is CREATED


# 6 - Download products

In [94]:
# order_id_CFA = 606417384 #Trial order finshied
# Use the API to get information about the order with ID order_id_CFA
order_id_CFA = order_info_CFA["id"]
order_info_CFA = requestAPI.get_order(order_id_CFA)

# Check the status of the order
if order_info_CFA['status'] == 'FINISHED':
    # If the order is finished, inform the user that it's ready for download
    print('The order {order_id} is finished and is ready for download'.format(order_id=order_id_CFA))
    
    # Get the products associated with this finished order
    order_products = requestAPI.get_order_products(order_id_CFA)
else:
    # If the order is not finished, inform the user of its status and that it cannot be downloaded
    print('The order {order_id} status {order_status}, it cannot be downloaded'.format(order_id=order_id_CFA, order_status=order_info_CFA['status']))


The order 606417384 is finished and is ready for download


In [99]:
order_products = requestAPI.get_order_products(606417384)
order_products

[{'jobId': 606423872,
  'productIdentifier': 'S3__20180801_000000_23680812_FISH_ARE_PFA',
  'type': 'wms-time-series',
  'layers': {'PFA': [{'title': '2018-08-01T00:00',
     'downloadURI': 'https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/S3__20180801_000000_23680812_FISH_ARE_PFA',
     'geoServerURI': 'https://data-lake.services4eo.com/geoserver/wms/reflect',
     'dateTime': '2018-08-01T00:00',
     'startTime': '2018-08-01T00:00:00',
     'stopTime': '2018-08-01T00:00:00',
     'wmsParams': {'format': 'image/png',
      'layers': 'shp_output_workspace:S3__20180801_000000_23680812_FISH_ARE_PFA_2018-08-01',
      'styles': 'S3_FISH_ARE_PFA,S3_FISH_ARE_FRONTS',
      'bbox': '37.337467,-11.052246,40.615198,-8.76709',
      'time': ''},
     'timestamps': []}]},
  'createdAt': '2023-09-07T08:50:21.185541',
  'updatedAt': '2023-09-07T08:50:21.185543'},
 {'jobId': 606423872,
  'productIdentifier': 'S3__20180801_000000_35301906_FISH_ARE_FRONTS',
  'typ

### Check the products we have to download

In [79]:
# Iterate through the order_products
for item in order_products:
    # Iterate through the layers in each order product
    for layer_name, layer_data in item['layers'].items():
        # Extract the download URI for the layer
        download_uri = layer_data[0]['downloadURI']
        
        # Print information about the layer and its download URI
        print(f"Layer: {layer_name}, Download URI: {download_uri}")

Layer: PFA, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/S3__20180801_000000_23680812_FISH_ARE_PFA
Layer: FRONTS, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/S3__20180801_000000_35301906_FISH_ARE_FRONTS


### Function download_products(folder, download_url)

This function is designed to download a file from a given URL and save it to the specified folder. It also includes error checking for HTTP status codes and prints a confirmation message upon successful download.

In [73]:
def download_products(folder, download_url):
    # Create the destination folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Extract the filename from the download URL
    filename = download_url.split("/")[-1]

    # Define the full path for the downloaded product
    downloaded_product = os.path.join(folder, f"{filename}.zip")

    # Send an HTTP GET request to the download URL
    with requests.get(download_url, stream=True, allow_redirects=True) as r:
        # Check for HTTP errors
        if r.status_code < 200 or r.status_code >= 300:
            print("HTTP ERROR %s - %s", r.status_code, r.text)

        # Write the received content to the local file
        with open(downloaded_product, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        # Print a confirmation message
        print('The product {product} has been downloaded into {folder}'.format(product=filename, folder=folder))
      

### Download the products available from the order

In [80]:
# Define the folder where you want to download the products
download_folder = 'OUTPUTS'

# Iterate through the order_products
for item in order_products:
    # Iterate through the layers in each order product
    for layer_name, layer_data in item['layers'].items():
        # Extract the download URI for the layer
        download_uri = layer_data[0]['downloadURI']
        
        # Print information about the layer and its download URI
        print(f"Layer: {layer_name}, Download URI: {download_uri}")
        
        # Call the download_products function to download the layer to the specified folder
        download_products(download_folder, download_uri)

Layer: PFA, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/S3__20180801_000000_23680812_FISH_ARE_PFA
The product S3__20180801_000000_23680812_FISH_ARE_PFA has been downloaded into OUTPUTS
Layer: FRONTS, Download URI: https://data-lake.services4eo.com/storage/b6443660-eede-4ec7-b026-68d0097d2ab8/output/S3__20180801_000000_35301906_FISH_ARE_FRONTS
The product S3__20180801_000000_35301906_FISH_ARE_FRONTS has been downloaded into OUTPUTS
